# IMPORTS

In [5]:
import time
import pyaudio
import wave
import torch
import librosa
import numpy as np
import soundfile as sf
from scipy.io import wavfile
from IPython.core.display import display
from IPython.display import Audio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer, Wav2Vec2Processor #PROCESSOR IS NOT BEING USED
from lang_trans.arabic import buckwalter
from datasets import load_dataset, load_metric
#import torchaudio

# Tokenizer and Model

In [6]:
tokenizer = Wav2Vec2Tokenizer.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-arabic")
model = Wav2Vec2ForCTC.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-arabic")

C:\anaconda\envs\nn\lib\site-packages\transformers\configuration_utils.py:370: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
C:\anaconda\envs\nn\lib\site-packages\transformers\models\wav2vec2\tokenization_wav2vec2.py:755: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Process

# Recording and Transcription

In [7]:
#function to countdown before starting to record
def countdown(t):
    print("starting to listen in", t, "seconds")
    while t:
        t -= 1
        mins, secs = divmod(t, 60)
        timer = '{:2d}'.format(secs)
        print(timer, end="\r")
        time.sleep(1)

In [8]:
#Function to record test speech
def listener(time):
    global filename
    chunk = 1024  # Record in chunks of 1024 samples
    sample_format = pyaudio.paInt16  # 16 bits per sample
    channels = 2
    fs = 44100  # Record at 44100 samples per second
    filename = "output.wav"

    p = pyaudio.PyAudio()  # Create an interface to PortAudio

    countdown(3)
    print('LISTENING...')
    stream = p.open(format=sample_format,
                    channels=channels,
                    rate=fs,
                    frames_per_buffer=chunk,
                    input=True)

    frames = []  # Initialize array to store frames
    
    count = 1
    for i in range(0, int(fs / chunk * time)):
        if(i%45 == 0):
            print(count)
            count = count+1
        data = stream.read(chunk)
        frames.append(data)

    # Stop and close the stream 
    stream.stop_stream() #to close recording
    stream.close() 
    # Terminate the PortAudio interface
    p.terminate() #to close pyaduio interface 

    print('FINISHED LISTENING')

    # Save the recorded data as a WAV file
    wf = wave.open(filename, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(sample_format))
    wf.setframerate(fs)
    wf.writeframes(b''.join(frames))
    wf.close()

In [19]:
listener(9)

print("\nSpeech Recording:")
display(Audio(filename))

input_audio, newframerate = librosa.load(filename, sr=16000)


input_values = tokenizer(input_audio, return_tensors="pt").input_values

logits = model(input_values).logits

predicted_ids = torch.argmax(logits, dim=-1)

tr = tokenizer.batch_decode(predicted_ids)[0]

print("Speech Transcription:")
buckwalter.untrans(tr)

starting to listen in 3 seconds
LISTENING...
1
2
3
4
5
6
7
8
9
FINISHED LISTENING

Speech Recording:


Speech Transcription:


'طعربيّ'

In [18]:
# input_audio

In [11]:
input_values

tensor([[-0.6456, -1.0774, -0.9441,  ...,  0.0059,  0.0059,  0.0000]])

In [12]:
# logits[0][0]

In [133]:
# predicted_ids

In [ ]:
# buckwalter.trans(tr) 